In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Health Data

In [4]:
r = requests.get(
    "https://www.cdc.gov/nchs/pressroom/sosmap/drug_poisoning_mortality/Drug_Overdose_dfe.json"
)

In [5]:
df = pd.DataFrame(r.json()["data"])

In [6]:
df["RATE"] = df["RATE"].astype(float)
df["DEATHS"] = df["DEATHS"].astype(int)

In [7]:
health_df = df[df["YEAR"] == "2014"].merge(
    df[df["YEAR"] == "2023"],
    on="STATE",
    how="inner",
    suffixes=("_2014", "_2023"),
)

In [12]:
health_df["CHANGE_RATE"] = (health_df["RATE_2023"] - health_df["RATE_2014"]).round(
    decimals=1
)
health_df["CHANGE_DEATHS"] = health_df["DEATHS_2023"] - health_df["DEATHS_2014"]

In [13]:
health_df = health_df.rename(columns={"STATE": "STUSPS"})

## Merge Data

In [14]:
health_gdf = states_df.merge(health_df, on="STUSPS", how="inner").dropna()

In [15]:
health_gdf = health_gdf.to_crs(9311)
health_gdf.to_file(
    "data/Increase_Drug_Overdose_Mortality_Rate_Per_State_2014-2023.gpkg"
)